# Multi AI Agent Systems with CrewAI - Complete Tutorial

A comprehensive guide to building production-ready multi-agent AI systems using CrewAI framework.

## Installation and Setup

Install all required packages for this tutorial:

In [ ]:
# Install required packages
%pip install crewai crewai_tools langchain_community setuptools

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

## Global Configuration

Set up API keys and environment variables used throughout the tutorial:

In [ ]:
import os

# Configure OpenAI API - Used for LLM agents throughout the tutorial
os.environ['OPENAI_API_KEY'] = '<your-openai-api-key>'
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

# Configure Serper API - Used for web search tools in later modules
# Get you key from https://serper.dev/
os.environ["SERPER_API_KEY"] = "<your-serper-api-key>"

# Import core CrewAI components used throughout tutorial
from crewai import Agent, Task, Crew

---
# 1. Course Overview

Welcome to this comprehensive course on Multi AI Agent Systems with CrewAI. You'll learn everything about agents, from basics to building complex production-ready systems.

## What You'll Learn
- **Role playing** and agent focus techniques
- **Tools** for agent communication with external world
- **Agent cooperation** and collaboration patterns
- **Guardrails** to prevent hallucinations and loops
- **Memory** systems for enhanced performance
- **Sequential, hierarchical, and asynchronous** workflows

## What You'll Build
1. **Research and Write Crew** - Simple content creation system
2. **Customer Support Crew** - Automated support system
3. **Customer Outreach Crew** - Sales automation system
4. **Event Planning System** - Complex coordination agents
5. **Financial Analysis System** - Advanced collaboration patterns
6. **Job Application Crew** - Production-ready resume optimization

The final project builds a crew that tailors resumes to job postings, increasing interview chances significantly.

---
# 2. Introduction to AI Agents and CrewAI

## What are AI Agents?
AI agents are LLMs that can talk to themselves with an inner thought process. They receive tasks and use tools to provide complex solutions beyond simple text generation.

## Why Multi-Agent Systems?
**Specialization**: One agent researches, another writes. Each focuses on their strength.

**Different Models**: Run researcher on Llama-3, writer on GPT-4, or use fine-tuned models.

**Better Results**: Focused agents outperform generalist agents trying to do everything.

## Why CrewAI?
CrewAI is an open-source framework designed for production use cases:

1. **Simple Structures** - Easy-to-understand concepts
2. **Assembly Pattern** - Predefined ways to connect systems
3. **Ready Tools** - Pre-built tools and integrations
4. **Custom Tools** - Framework for building your own
5. **Production Platform** - Deploy agents to production

Core building blocks: **Agents**, **Tasks**, and **Crews**.

---
# 3. Create Agents to Research and Write an Article

Build your first multi-agent system with three specialized agents working together to create high-quality content.

**Key Learning**: Agents perform better when role-playing with clear roles, goals, and backstories.

## Creating Agents

Define agents with `role`, `goal` and `backstory` for better LLM performance.

In [ ]:
# AGENT CREATION: Demonstrates the three core elements of effective agents
# 1. Role: Specific job function
# 2. Goal: Clear objective with placeholder for dynamic input
# 3. Backstory: Context that helps LLM understand the agent's perspective
planner = Agent(
    role="Content Planner",  # Specialized role for content strategy
    goal="Plan engaging and factually accurate content on {topic}",  # Dynamic goal using input variables
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,  # COOPERATION: Prevents this agent from delegating to others
	verbose=True  # DEBUGGING: Shows agent thinking process
)

In [ ]:
# ROLE PLAYING: Writer agent with specific expertise and dependencies
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

In [ ]:
# QUALITY CONTROL: Editor agent for final review and refinement
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

Define tasks with `description`, `expected_output` and assigned `agent`.

In [ ]:
# TASK DESIGN: Clear description with numbered steps and specific deliverables
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"  # Step-by-step instructions
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",  # CLEAR EXPECTATIONS: Specific deliverable format
    agent=planner,  # AGENT ASSIGNMENT: Links task to specific agent
)

In [ ]:
# SEQUENTIAL WORKFLOW: This task depends on the planner's output
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

In [ ]:
# FINAL REVIEW: Quality assurance step in the pipeline
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

Crews execute tasks sequentially by default. Task order matters since output flows to the next task.

In [ ]:
# SEQUENTIAL PROCESS: Default execution pattern where tasks run one after another
# Task order is critical - output of each task becomes input for the next
crew = Crew(
    agents=[planner, writer, editor],  # Agent pool available to execute tasks
    tasks=[plan, write, edit],  # Tasks executed in order: plan -> write -> edit
    verbose=2  # LOGGING: Level 2 shows detailed execution logs
)

## Running the Crew

In [ ]:
# EXECUTION: Kick off the crew with input variables
# The {topic} placeholder in agent goals and task descriptions gets replaced
result = crew.kickoff(inputs={"topic": "AI Agents"})

In [ ]:
# OUTPUT DISPLAY: Render the final result as formatted markdown
from IPython.display import Markdown
Markdown(result)

---
# 4. Key Elements of AI Agents

Learn the six key elements that make agents perform better: **Role Playing**, **Focus**, **Tools**, **Cooperation**, **Guardrails**, and **Memory**.

Build a customer support automation system demonstrating these principles.

## Role Playing, Focus and Cooperation

In [ ]:
# ROLE PLAYING: Specific customer support role with detailed context
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,  # FOCUS: This agent cannot delegate, must handle requests directly
	verbose=True
)

In [ ]:
# COOPERATION: Quality assurance agent can work with support agent
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True
    # allow_delegation=True by default - this agent CAN delegate back to support agent
)

## Tools, Guardrails and Memory

### Tools
Tools allow agents to communicate with the external world and perform actions beyond text generation.

In [ ]:
# TOOLS: Import CrewAI's pre-built tool collection
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool

In [ ]:
# SPECIALIZED TOOL: Website scraper configured for specific documentation
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

### Creating Tasks

Tools can be assigned at Agent level (all tasks) or Task level (specific task only).

In [ ]:
# TASK-LEVEL TOOLS: Tools assigned to specific task only
# This demonstrates tool assignment at task level vs agent level
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"  # Dynamic input variables for personalization
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],  # TOOL ASSIGNMENT: Only this task can use this tool
    agent=support_agent,
)

In [ ]:
# COOPERATION PATTERN: QA agent reviews work without using tools
# This shows agent collaboration where one agent reviews another's work
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry."
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
    # No tools assigned - this agent works with the previous agent's output
)

### Creating the Crew

#### Memory
Setting `memory=True` enables the crew to remember and learn from interactions.

In [ ]:
# MEMORY: Enables crew to remember and learn from past interactions
# This is one of the six key elements that makes agents better
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=2,
  memory=True  # MEMORY SYSTEM: Crew remembers context across executions
)

### Running the Crew

#### Guardrails
Agents stay within expected scope and provide reliable responses.

In [ ]:
# GUARDRAILS: Input validation and structured execution prevent agent derailment
# CrewAI automatically implements guardrails to prevent loops and hallucinations
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)

In [ ]:
from IPython.display import Markdown
Markdown(result)

---
# 5. Mental Framework for Agent Creation

## Think Like a Manager
The key to creating great multi-agent systems is thinking like a manager. There's a high correlation between good management skills and effective agent design.

## Framework Steps

1. **Define the Goal**: What exactly are you trying to accomplish?
2. **Map the Process**: What process should agents follow?
3. **Identify People**: What people would you hire to get this job done?
4. **Define Roles**: What should be their roles, goals, and backstories?

## Better Examples
Instead of generic roles, use specific expertise:

- **Bad**: "Researcher" → **Good**: "HR Research Specialist"
- **Bad**: "Writer" → **Good**: "Senior Copywriter"
- **Bad**: "Financial Analyst" → **Good**: "FINRA Approved Analyst"

## Key Takeaways
- Agents perform better with **role-playing**
- **Focus** beats generalization
- **Cooperation** enables complex workflows
- **Guardrails** prevent derailment
- **Memory** enables self-improvement
- **Smaller, focused tasks** outperform large, complex ones

---
# 6. Key Elements of Agent Tools

Tools are crucial for agents to communicate with the external world and have real impact. Great tools have three key characteristics:

## 1. Versatility
Tools must handle different types of requests from LLMs. They bridge fuzzy AI inputs with strong-typed external systems. CrewAI automatically converts arguments to correct types.

## 2. Fault Tolerance
Tools should fail gracefully without stopping execution. When exceptions occur, CrewAI sends error messages back to agents so they can adjust and retry rather than crashing.

This is crucial when deploying at scale - agents deal with various documents and data sources, and tools must handle unexpected inputs.

## 3. Caching
Smart caching prevents unnecessary API calls and improves performance. CrewAI offers cross-agent caching where multiple agents can benefit from cached results.

Caching is key for real-world usability when tools frequently call external APIs and services.

---
# 7. Tools for Customer Outreach Campaign

Build a sales automation system using CrewAI tools and custom tools for customer outreach.

**Key Learning**: Tools enable agents to take actions beyond text generation.

## Creating Agents

In [ ]:
# SPECIALIZED SALES AGENT: Demonstrates focused agent design for lead generation
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is foundational to the "
        "sales process, setting the stage for meaningful "
        "engagements and successful conversions."
    ),
    allow_delegation=False,
	verbose=True
)

In [ ]:
# COMMUNICATION SPECIALIST: Focused on personalized outreach and messaging
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling outreach",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating personalized outreach messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
	verbose=True
)

## crewAI Tools

In [ ]:
# TOOL VARIETY: Different types of tools for different agent needs
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

# DIRECTORY TOOL: Allows agents to read instruction files for different customer types
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()  # VERSATILITY: Can read any file dynamically
search_tool = SerperDevTool()    # WEB SEARCH: Enables internet research capabilities

## Creating a Custom Tool

In [ ]:
# CUSTOM TOOL FRAMEWORK: How to build tools that extend agent capabilities
from crewai_tools import BaseTool

In [ ]:
# CUSTOM TOOL IMPLEMENTATION: Demonstrates tool creation pattern
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"  # TOOL IDENTIFICATION: Clear name for agent selection
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")  # TOOL PURPOSE: When agents should use this
    
    def _run(self, text: str) -> str:
        # FAULT TOLERANCE: In production, this would handle various text inputs
        # and gracefully handle errors instead of crashing
        return "positive"  # Simplified implementation for demonstration

In [ ]:
# TOOL INSTANTIATION: Create instance for use by agents
sentiment_analysis_tool = SentimentAnalysisTool()

## Creating Tasks

In [ ]:
# RESEARCH TASK: Demonstrates multiple tool usage for comprehensive analysis
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],  # MULTIPLE TOOLS: Agent can choose best tool for each subtask
    agent=sales_rep_agent,
)

In [ ]:
# PERSONALIZATION TASK: Uses custom tool for sentiment analysis
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {key_decision_maker_position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling subject line, personalized greeting, "
        "a body that addresses their recent milestone {milestone}, "
        "explains how our solutions align with their goals, "
        "and a clear call-to-action. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate culture."
    ),
    tools=[sentiment_analysis_tool, search_tool],  # CUSTOM + STANDARD TOOLS: Mix of custom and pre-built tools
    agent=lead_sales_rep_agent,
)

## Creating the Crew

In [ ]:
# SALES WORKFLOW: Sequential process where research informs outreach
crew = Crew(
    agents=[sales_rep_agent, 
            lead_sales_rep_agent],
    
    tasks=[lead_profiling_task, 
           personalized_outreach_task],  # TASK DEPENDENCY: Second task uses first task's output
	
    verbose=2,
	memory=True  # LEARNING: Crew remembers successful outreach patterns
)

## Running the Crew

In [ ]:
# DYNAMIC INPUT: Variables allow reusing the crew for different leads
# Note: If you see an error like "Action Input: {"file_path": "DeepLearningAI_company_profile.txt"}", 
# it's just the agent attempting to use the file reader tool for a file that may not exist. 
# This is expected behavior and not a problem.
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "key_decision_maker_position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

In [ ]:
from IPython.display import Markdown
Markdown(result)

---
# 8. Key Elements of Well-Defined Tasks

Tasks are the cornerstone of multi-agent systems. Just like managing people, you need to clearly delegate work with explicit expectations.

## Manager Analogy Extended
When hiring someone, especially junior people, you must be explicit about:
- **What you expect them to do**
- **What the expected result looks like**

## Task Design Principles

### Two Essential Elements
1. **Clear Description** - What is the task exactly?
2. **Expected Output** - What should the result look like?

### Best Practices
- Break complex work into **smaller, focused tasks**
- Each task should have a **single clear objective**
- Define **specific deliverables** and formats
- Consider **task dependencies** and order
- Assign tasks to **appropriate specialist agents**

## Why This Matters
- **Smaller tasks** with **focused agents** perform better than large, complex tasks
- Clear expectations reduce ambiguity and improve results
- Well-defined outputs enable smooth handoffs between agents
- Focused tasks allow agents to specialize and excel

---
# 9. Automate Event Planning

Build a multi-agent system for comprehensive event planning, demonstrating advanced task coordination.

**Key Learning**: Complex coordination through specialized agents and well-defined task flows.

## crewAI Tools

In [ ]:
# RESEARCH TOOLS: Web-based tools for event planning research
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools for market research and venue investigation
search_tool = SerperDevTool()    # Web search for venue and vendor discovery
scrape_tool = ScrapeWebsiteTool()  # Deep dive into specific venue websites

## Creating Agents

In [ ]:
# SPECIALIZED COORDINATOR: Domain-specific agent for venue management
# Demonstrates how to create focused agents for complex multi-step processes
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],  # AGENT-LEVEL TOOLS: Available for all tasks this agent performs
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    )
)

In [ ]:
# OPERATIONS SPECIALIST: Handles complex logistical coordination
logistics_coordinator = Agent(
    role="Logistics Coordinator",
    goal="Manage all logistics for the event "
    "including catering, equipment, and transportation",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect "
        "of the event is planned and executed flawlessly. "
        "From catering to equipment setup, "
        "you handle it all with precision."
    )
)

In [ ]:
# COMMUNICATIONS EXPERT: Manages audience engagement and promotion
marketing_communications_coordinator = Agent(
    role="Marketing and Communications Coordinator",
    goal="Promote the event and communicate "
    "with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "develop marketing strategies that "
        "resonate with the target audience. "
        "Your goal is to maximize event attendance "
        "and engagement through effective communication."
    )
)

## Creating Tasks

In [ ]:
# TASK COORDINATION: Each task builds foundation for subsequent tasks
# Venue selection affects logistics and marketing strategies
venue_task = Task(
    description=(
        "Find a venue in {event_city} "
        "that meets criteria for {event_topic}."
    ),
    expected_output=(
        "A detailed venue report including name, location, "
        "capacity, cost, amenities, and "
        "contact information of the venue."
    ),
    agent=venue_coordinator  # AGENT SPECIALIZATION: Right agent for the right task
)

In [ ]:
# DEPENDENCY TASK: Uses venue information to plan logistics
# Shows how task outputs flow to inform subsequent decisions
logistics_task = Task(
    description=(
        "Coordinate catering and "
        "equipment for an event "
        "with the following details:\n"
        "- Event topic: {event_topic}\n"
        "- Number of participants: {expected_participants}\n"
        "- Venue: {venue}\n"  # This will be filled by venue_task output
        "- Location: {event_city}\n"
    ),
    expected_output=(
        "A comprehensive logistics report including "
        "catering options, equipment needs, "
        "setup requirements, and contact information "
        "for all vendors and service providers."
    ),
    agent=logistics_coordinator
)

In [ ]:
# PARALLEL EXECUTION: Marketing can run alongside logistics planning
# Demonstrates how some tasks can be independent
marketing_task = Task(
    description=(
        "Promote the {event_topic} event "
        "aiming to engage at least "
        "{expected_participants} potential participants.\n"
        "Use social media, email campaigns, "
        "and other communication channels."
    ),
    expected_output=(
        "A marketing plan including key messages, "
        "promotional materials, communication channels, "
        "timeline, and expected reach and engagement metrics."
    ),
    agent=marketing_communications_coordinator
)

## Creating the Crew

In [ ]:
# COMPLEX COORDINATION: Multiple specialists working in sequence
# Task order ensures dependencies are resolved before dependent tasks execute
event_management_crew = Crew(
    agents=[venue_coordinator, 
            logistics_coordinator, 
            marketing_communications_coordinator],
    
    tasks=[venue_task, 
           logistics_task, 
           marketing_task],  # SEQUENTIAL EXECUTION: Venue -> Logistics -> Marketing
    
    verbose=2  # MONITORING: Track complex multi-agent coordination
)

## Running the Crew

In [ ]:
# COMPREHENSIVE INPUT: All parameters needed for complete event planning
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_city': "San Francisco", 
    'expected_participants': 500,
    'venue': "Golden Gate Park"  # This may be updated by venue_coordinator
}

# COMPLEX WORKFLOW EXECUTION: Multiple agents coordinating specialized tasks
result = event_management_crew.kickoff(inputs=event_details)

In [ ]:
from IPython.display import Markdown
Markdown(result)

---
# 10. Multi-Agent Collaboration Patterns

Learn different ways agents can collaborate beyond simple sequential execution.

## Collaboration Patterns

### Sequential Process (Default)
One agent finishes a task, passes output to the next. Simple but context can fade over long chains.

### Parallel Process
Multiple agents work simultaneously on different tasks. Efficient for independent workstreams.

### Hierarchical Process
A manager agent coordinates and delegates work to specialist agents. Powerful for complex scenarios requiring oversight.

## When to Use Each Pattern

- **Sequential**: Linear workflows, content creation pipelines
- **Parallel**: Independent research streams, multi-channel analysis
- **Hierarchical**: Complex decision-making, dynamic task allocation

## Real-World Applications
Fortune 500 companies use CrewAI for financial analysis, combining multiple collaboration patterns to:
- Research market trends
- Analyze company fundamentals  
- Generate trading recommendations
- Perform risk assessments

The key is choosing the right collaboration pattern for your specific use case.

---
# 11. Multi-Agent Collaboration for Financial Analysis

Build an advanced financial analysis system demonstrating hierarchical collaboration with a manager agent.

**Key Learning**: Hierarchical processes with autonomous delegation and complex workflows.

## crewAI Tools

In [ ]:
# FINANCIAL RESEARCH TOOLS: Specialized tools for market analysis
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()     # Market data and news search
scrape_tool = ScrapeWebsiteTool() # Deep analysis of financial websites

## Creating Agents

In [ ]:
# QUANTITATIVE SPECIALIST: Data-focused agent for market analysis
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    tools=[search_tool, scrape_tool]  # RESEARCH CAPABILITIES: Access to market data sources
)

In [ ]:
# STRATEGY SPECIALIST: Translates analysis into actionable strategies
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    tools=[search_tool, scrape_tool]
)

In [ ]:
# EXECUTION SPECIALIST: Focuses on implementation details
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    tools=[search_tool, scrape_tool]
)

In [ ]:
# RISK SPECIALIST: Critical oversight for financial safety
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading decisions.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It provides a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm's risk tolerance.",
    verbose=True,
    tools=[search_tool, scrape_tool]
)

## Creating Tasks

In [ ]:
# FOUNDATIONAL ANALYSIS: Data gathering and trend identification
# This task feeds into all other financial analysis tasks
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
    # This task can run independently and inform others
)

In [ ]:
# STRATEGY DEVELOPMENT: Builds on data analysis to create actionable plans
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
    # Depends on data_analysis_task output
)

In [ ]:
# IMPLEMENTATION PLANNING: Tactical execution of strategies
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine optimal "
        "execution methods. Provide detailed execution plans "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute the proposed trading strategies."
    ),
    agent=execution_agent,
    # Depends on strategy_development_task
)

In [ ]:
# RISK OVERSIGHT: Critical review of all proposed actions
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans. Provide a comprehensive "
        "risk analysis and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for the proposed "
        "trading strategies."
    ),
    agent=risk_management_agent,
    # Reviews output from all other tasks
)

## Creating the Crew

Using hierarchical process with manager delegation.

In [ ]:
# HIERARCHICAL PROCESS: Manager agent coordinates specialist agents
# This demonstrates advanced collaboration pattern for complex workflows
from crewai import Process
from langchain_openai import ChatOpenAI

financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    # HIERARCHICAL CONFIGURATION: Manager LLM oversees and delegates
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),  # Manager decision-making model
    process=Process.hierarchical,  # ADVANCED COLLABORATION: Manager delegates work dynamically
    verbose=2  # MONITORING: Track manager delegation decisions
)

## Running the Crew

In [ ]:
# COMPREHENSIVE FINANCIAL PARAMETERS: All inputs for complex analysis
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

# HIERARCHICAL EXECUTION: Manager agent coordinates all specialist agents
# Watch for delegation patterns and autonomous decision-making
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

In [ ]:
from IPython.display import Markdown
Markdown(result)

---
# 12. Build a Crew to Tailor Job Applications

Build the most complex and practical crew: a system that analyzes job postings and tailors resumes to increase interview chances.

**Key Learning**: Production-ready system combining all concepts learned.

## crewAI Tools

In [ ]:
# COMPREHENSIVE TOOL SUITE: Multiple tool types for complex job application workflow
from crewai_tools import (
  FileReadTool,       # Read resume files
  ScrapeWebsiteTool,  # Analyze job postings
  MDXSearchTool,      # Semantic search through resume
  SerperDevTool       # Research companies and roles
)

search_tool = SerperDevTool()  # Company and role research
scrape_tool = ScrapeWebsiteTool()  # Job posting analysis
read_resume = FileReadTool(file_path='./resume/fake_resume.md')  # Resume content access
semantic_search_resume = MDXSearchTool(mdx='./resume/fake_resume.md')  # Smart resume search

## Creating Agents

In [ ]:
# JOB ANALYSIS SPECIALIST: Extracts requirements from job postings
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],  # WEB RESEARCH: Job posting and company analysis
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)

In [ ]:
# CANDIDATE ANALYSIS SPECIALIST: Comprehensive applicant profiling
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],  # MULTI-SOURCE: Web + file-based research
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

In [ ]:
# RESUME OPTIMIZATION SPECIALIST: Strategic resume enhancement
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

In [ ]:
# INTERVIEW PREPARATION SPECIALIST: Final stage optimization
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

## Creating Tasks

In [ ]:
# PARALLEL EXECUTION: This task can run independently
# async_execution=True allows this to run while other tasks execute
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True  # PARALLEL PROCESSING: Can run simultaneously with profile_task
)

In [ ]:
# PARALLEL EXECUTION: Independent candidate analysis
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True  # PARALLEL PROCESSING: Can run simultaneously with research_task
)

In [ ]:
# SYNTHESIS TASK: Combines outputs from parallel tasks
# context=[research_task, profile_task] ensures this waits for both to complete
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update the resume to better "
        "reflect the candidates abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",  # FILE OUTPUT: Saves result to disk
    context=[research_task, profile_task],  # TASK DEPENDENCIES: Waits for both parallel tasks
    agent=resume_strategist
)

In [ ]:
# FINAL PREPARATION: Uses all previous outputs
# context includes all previous tasks - demonstrates complex workflow coordination
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Prepare questions and talking points that help the candidate "
        "highlight the main points of the resume and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",  # FILE OUTPUT: Creates interview prep document
    context=[research_task, profile_task, resume_strategy_task],  # ALL DEPENDENCIES: Uses entire workflow output
    agent=interview_preparer
)

## Creating the Crew

In [ ]:
# PRODUCTION WORKFLOW: Complex coordination with parallel and sequential elements
# Demonstrates advanced crew design for real-world applications
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,                    # PARALLEL: Runs with research_task
           resume_strategy_task,            # SEQUENTIAL: Waits for both parallel tasks
           interview_preparation_task],     # SEQUENTIAL: Waits for all previous tasks

    verbose=2  # PRODUCTION MONITORING: Track complex workflow execution
)

## Running the Crew

In [ ]:
# PRODUCTION INPUT: Real-world job application scenario
# This demonstrates how to structure inputs for complex, multi-variable workflows
job_application_inputs = {
    'job_posting_url': 'https://jobs.lever.co/FiscalNote/6d7f5025-50b4-493d-b6aa-9689c699c267',
    'github_url': 'https://github.com/joaomdmoura',
    'personal_writeup': """Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his ability to drive
    innovation and growth in the tech industry. Ideal for leadership
    roles that require a strategic and innovative approach."""
}

In [ ]:
# COMPLEX EXECUTION: This will take several minutes due to the sophisticated workflow
# Watch for parallel execution, task dependencies, and file generation
result = job_application_crew.kickoff(inputs=job_application_inputs)

## View Generated Files

In [ ]:
# FILE OUTPUT REVIEW: Display the tailored resume generated by the crew
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

In [ ]:
# INTERVIEW PREPARATION: Display the interview materials created for the candidate
display(Markdown("./interview_materials.md"))

---
# Congratulations!

You've completed the Multi AI Agent Systems with CrewAI tutorial. You now know how to:

**Build multi-agent systems** with specialized roles  
**Create effective agents** using role-playing and backstories  
**Design focused tasks** with clear expectations  
**Implement tools** for external world interaction  
**Apply collaboration patterns** (sequential, parallel, hierarchical)  
**Use memory and guardrails** for robust systems  
**Build production-ready** applications  

## Resources

- [CrewAI Documentation](https://docs.crewai.com)
- [Community Forum](https://community.crewai.com)
